# Mental Health Tweet Classification - Exploratory Data Analysis

This notebook performs comprehensive exploratory data analysis on the mental health tweet dataset to understand:
- Dataset structure and class balance
- Text characteristics and distributions
- Language patterns between classes
- Visual patterns in the data

**Dataset**: Depression vs Non-Depression Tweet Classification
**Date**: September 30, 2025

In [8]:
# Import libraries
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from collections import Counter
import re
import json
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# Import our custom modules
import sys
sys.path.append('..')
from src.data.load_data import DataLoader
from src.data.preprocess import TweetPreprocessor, FeatureExtractor

# Set plotting style
plt.style.use('seaborn-v0_8')
sns.set_palette("husl")

# Create output directories
Path('../reports').mkdir(exist_ok=True)
Path('../reports/figures').mkdir(exist_ok=True)

print("Libraries imported successfully!")

Libraries imported successfully!


## 1. Data Loading and Schema Overview

In [9]:
# Initialize data loader
loader = DataLoader(dataset_dir='../dataset')

# Load schema information
schema = loader.load_schema()
print("Dataset Schema Summary:")
print(json.dumps(schema['dataset_analysis']['summary'], indent=2))

Dataset Schema Summary:


KeyError: 'dataset_analysis'

In [ ]:
# Quick data loading and exploration
import pandas as pd
import os

# Load the dataset files directly
dataset_dir = "../dataset"
files = os.listdir(dataset_dir)
print("Dataset files found:")
for f in files:
    if f.endswith('.csv'):
        print(f"  - {f}")

# Load and examine the first file
if 'd_tweets.csv' in files:
    df_depression = pd.read_csv(os.path.join(dataset_dir, 'd_tweets.csv'))
    print(f"\n📊 Depression tweets dataset:")
    print(f"   Shape: {df_depression.shape}")
    print(f"   Columns: {list(df_depression.columns)}")
    print(f"\nFirst few rows:")
    print(df_depression.head(3))

if 'non_d_tweets.csv' in files:
    df_non_depression = pd.read_csv(os.path.join(dataset_dir, 'non_d_tweets.csv'))
    print(f"\n📊 Non-depression tweets dataset:")
    print(f"   Shape: {df_non_depression.shape}")
    print(f"   Columns: {list(df_non_depression.columns)}")
    print(f"\nFirst few rows:")
    print(df_non_depression.head(3))

Dataset files found:
  - clean_d_tweets.csv
  - clean_non_d_tweets.csv
  - d_tweets.csv
  - non_d_tweets.csv

📊 Depression tweets dataset:
   Shape: (3496, 38)
   Columns: ['id', 'conversation_id', 'created_at', 'date', 'timezone', 'place', 'tweet', 'language', 'hashtags', 'cashtags', 'user_id', 'user_id_str', 'username', 'name', 'day', 'hour', 'link', 'urls', 'photos', 'video', 'thumbnail', 'retweet', 'nlikes', 'nreplies', 'nretweets', 'quote_url', 'search', 'near', 'geo', 'source', 'user_rt_id', 'user_rt', 'retweet_id', 'reply_to', 'retweet_date', 'translate', 'trans_src', 'trans_dest']

First few rows:
                    id      conversation_id    created_at  \
0  1261623711502753792  1261623711502753792  1.589630e+12   
1  1255537912491343872  1255537912491343872  1.588179e+12   
2  1255524270521761792  1255524270521761792  1.588175e+12   

                  date  timezone  place  \
0  2020-05-16 01:45:07     -1000    NaN   
1  2020-04-29 06:42:19     -1000    NaN   
2  2020-04-29

In [ ]:
# Create combined dataset for analysis
import numpy as np

# Load all datasets and combine them
datasets = {}

# Load depression and non-depression tweets
if 'd_tweets.csv' in files:
    df_d = pd.read_csv(os.path.join(dataset_dir, 'd_tweets.csv'))
    df_d['label'] = 1  # Depression = 1
    df_d['label_name'] = 'depression'
    datasets['depression'] = df_d
    print(f"✅ Loaded {len(df_d):,} depression tweets")

if 'non_d_tweets.csv' in files:
    df_nd = pd.read_csv(os.path.join(dataset_dir, 'non_d_tweets.csv'))  
    df_nd['label'] = 0  # Non-depression = 0
    df_nd['label_name'] = 'non_depression'
    datasets['non_depression'] = df_nd
    print(f"✅ Loaded {len(df_nd):,} non-depression tweets")

# Combine datasets
if datasets:
    # Find common columns
    common_cols = set(datasets[list(datasets.keys())[0]].columns)
    for df in datasets.values():
        common_cols = common_cols.intersection(set(df.columns))
    
    print(f"\nCommon columns: {list(common_cols)}")
    
    # Combine datasets using common columns
    combined_dfs = []
    for name, df in datasets.items():
        df_subset = df[list(common_cols)].copy()
        print(f"{name}: {len(df_subset)} rows")
        combined_dfs.append(df_subset)
    
    # Create final combined dataset
    df_combined = pd.concat(combined_dfs, ignore_index=True)
    print(f"\n📊 Combined dataset shape: {df_combined.shape}")
    print(f"📊 Class distribution:")
    print(df_combined['label_name'].value_counts())
    
    # Show sample data
    print(f"\n📋 Sample data:")
    for label in [0, 1]:
        sample = df_combined[df_combined['label'] == label].iloc[0]
        label_name = sample['label_name']
        # Find text column
        text_cols = [col for col in df_combined.columns if 'tweet' in col.lower() or 'text' in col.lower()]
        if text_cols:
            text = str(sample[text_cols[0]])[:100] + "..." if len(str(sample[text_cols[0]])) > 100 else str(sample[text_cols[0]])
            print(f"  {label_name}: {text}")
else:
    print("❌ No datasets loaded successfully")

✅ Loaded 3,496 depression tweets
✅ Loaded 4,809 non-depression tweets

Common columns: ['label', 'conversation_id', 'thumbnail', 'user_id_str', 'tweet', 'user_id', 'reply_to', 'translate', 'label_name', 'quote_url', 'hashtags', 'retweet_id', 'nreplies', 'photos', 'search', 'link', 'created_at', 'id', 'geo', 'user_rt', 'cashtags', 'trans_dest', 'video', 'day', 'trans_src', 'urls', 'retweet', 'language', 'nlikes', 'date', 'retweet_date', 'name', 'place', 'user_rt_id', 'username', 'near', 'hour', 'source', 'timezone', 'nretweets']
depression: 3496 rows
non_depression: 4809 rows

📊 Combined dataset shape: (8305, 40)
📊 Class distribution:
label_name
non_depression    4809
depression        3496
Name: count, dtype: int64

📋 Sample data:
  non_depression: Touch Passes are the new ASMR
  depression: the real reason why you're sad? you're attached to people who have been distant with you. you're pay...


In [ ]:
# Quick baseline model training
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import joblib

print("🤖 Training Baseline Model...")

# Prepare the data
X = df_combined['tweet'].fillna('')  # Use tweet text as features
y = df_combined['label']  # Use binary labels

print(f"Dataset: {len(X)} samples")
print(f"Classes: {np.bincount(y)} (0: non-depression, 1: depression)")

# Split the data
X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

print(f"Training set: {len(X_train)} samples")
print(f"Test set: {len(X_test)} samples")

# Create TF-IDF features
print("\n🔧 Creating TF-IDF features...")
vectorizer = TfidfVectorizer(
    max_features=10000,
    stop_words='english',
    ngram_range=(1, 2),
    min_df=2,
    max_df=0.95
)

X_train_tfidf = vectorizer.fit_transform(X_train)
X_test_tfidf = vectorizer.transform(X_test)

print(f"Feature matrix shape: {X_train_tfidf.shape}")

# Train Logistic Regression model
print("\n🎯 Training Logistic Regression...")
model = LogisticRegression(
    class_weight='balanced',
    random_state=42,
    max_iter=1000
)

model.fit(X_train_tfidf, y_train)

# Make predictions
y_pred = model.predict(X_test_tfidf)
y_pred_proba = model.predict_proba(X_test_tfidf)

# Evaluate model
accuracy = accuracy_score(y_test, y_pred)
print(f"\n📊 Model Performance:")
print(f"Accuracy: {accuracy:.4f}")

print(f"\nClassification Report:")
print(classification_report(y_test, y_pred, target_names=['Non-Depression', 'Depression']))

print(f"\nConfusion Matrix:")
cm = confusion_matrix(y_test, y_pred)
print(cm)

🤖 Training Baseline Model...
Dataset: 8305 samples
Classes: [4809 3496] (0: non-depression, 1: depression)
Training set: 6644 samples
Test set: 1661 samples

🔧 Creating TF-IDF features...
Feature matrix shape: (6644, 8491)

🎯 Training Logistic Regression...

📊 Model Performance:
Accuracy: 0.8561

Classification Report:
                precision    recall  f1-score   support

Non-Depression       0.88      0.88      0.88       962
    Depression       0.83      0.83      0.83       699

      accuracy                           0.86      1661
     macro avg       0.85      0.85      0.85      1661
  weighted avg       0.86      0.86      0.86      1661


Confusion Matrix:
[[843 119]
 [120 579]]


In [ ]:
# Save the model and vectorizer
import os
os.makedirs('../models/baseline', exist_ok=True)

print("💾 Saving model...")
joblib.dump(model, '../models/baseline/logistic_model.joblib')
joblib.dump(vectorizer, '../models/baseline/tfidf_vectorizer.joblib')

print("✅ Model saved to models/baseline/")

# Test model on some sample predictions
print("\n🧪 Testing model on sample texts:")

test_texts = [
    "I feel so sad and hopeless, nothing matters anymore",
    "Having a great day, feeling amazing!",
    "Just finished a workout, feeling energized",
    "Everything is falling apart, I can't take it anymore"
]

for i, text in enumerate(test_texts, 1):
    # Transform text using our vectorizer
    text_tfidf = vectorizer.transform([text])
    
    # Get prediction and probability
    pred = model.predict(text_tfidf)[0]
    prob = model.predict_proba(text_tfidf)[0]
    
    label_name = "Depression" if pred == 1 else "Non-Depression"
    confidence = max(prob) * 100
    
    print(f"\n{i}. Text: \"{text[:50]}{'...' if len(text) > 50 else ''}\"")
    print(f"   Prediction: {label_name} (confidence: {confidence:.1f}%)")

print(f"\n🎉 Baseline model training completed successfully!")
print(f"📋 Summary:")
print(f"   - Model type: Logistic Regression with TF-IDF")
print(f"   - Training samples: {len(X_train):,}")
print(f"   - Test accuracy: {accuracy:.4f}")
print(f"   - Model saved to: models/baseline/")

💾 Saving model...
✅ Model saved to models/baseline/

🧪 Testing model on sample texts:

1. Text: "I feel so sad and hopeless, nothing matters anymor..."
   Prediction: Depression (confidence: 93.5%)

2. Text: "Having a great day, feeling amazing!"
   Prediction: Non-Depression (confidence: 59.8%)

3. Text: "Just finished a workout, feeling energized"
   Prediction: Depression (confidence: 72.5%)

4. Text: "Everything is falling apart, I can't take it anymo..."
   Prediction: Depression (confidence: 91.3%)

🎉 Baseline model training completed successfully!
📋 Summary:
   - Model type: Logistic Regression with TF-IDF
   - Training samples: 6,644
   - Test accuracy: 0.8561
   - Model saved to: models/baseline/


In [ ]:
# Load raw data
print("Loading raw tweet data...")
texts_raw, labels_raw = loader.get_text_and_labels(use_clean=False)

print(f"Raw dataset: {len(texts_raw)} samples")
print(f"Class distribution: {pd.Series(labels_raw).value_counts().to_dict()}")

# Create DataFrame for analysis
df_raw = pd.DataFrame({
    'text': texts_raw,
    'label': labels_raw,
    'class_name': ['Depression' if label == 1 else 'Non-Depression' for label in labels_raw]
})

print("\nDataset loaded successfully!")
print(f"Shape: {df_raw.shape}")
print(f"Columns: {df_raw.columns.tolist()}")

## 2. Class Balance Analysis

In [ ]:
# Class distribution
class_counts = df_raw['class_name'].value_counts()
class_props = df_raw['class_name'].value_counts(normalize=True) * 100

print("Class Distribution:")
for class_name in class_counts.index:
    count = class_counts[class_name]
    prop = class_props[class_name]
    print(f"  {class_name}: {count:,} samples ({prop:.1f}%)")

# Visualize class distribution
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(12, 5))

# Bar plot
class_counts.plot(kind='bar', ax=ax1, color=['#ff7f7f', '#7f7fff'])
ax1.set_title('Class Distribution (Count)')
ax1.set_ylabel('Number of Tweets')
ax1.set_xlabel('Class')
ax1.tick_params(axis='x', rotation=45)

# Pie chart
ax2.pie(class_counts.values, labels=class_counts.index, autopct='%1.1f%%', 
        colors=['#ff7f7f', '#7f7fff'], startangle=90)
ax2.set_title('Class Distribution (Proportion)')

plt.tight_layout()
plt.savefig('../reports/figures/class_distribution.png', dpi=300, bbox_inches='tight')
plt.show()

# Check for class imbalance
imbalance_ratio = class_counts.max() / class_counts.min()
print(f"\nClass imbalance ratio: {imbalance_ratio:.2f}")
if imbalance_ratio > 2:
    print("⚠️  Dataset is imbalanced. Consider using class weights or resampling.")
else:
    print("✅ Dataset is reasonably balanced.")

## 3. Text Length and Basic Statistics

In [ ]:
# Calculate text statistics
df_raw['text_length'] = df_raw['text'].str.len()
df_raw['word_count'] = df_raw['text'].str.split().str.len()
df_raw['sentence_count'] = df_raw['text'].str.count(r'[.!?]+') + 1

# Summary statistics by class
print("Text Length Statistics by Class:")
text_stats = df_raw.groupby('class_name')[['text_length', 'word_count', 'sentence_count']].describe()
print(text_stats.round(2))

# Visualize text length distributions
fig, axes = plt.subplots(2, 2, figsize=(15, 10))

# Character length distribution
for class_name in df_raw['class_name'].unique():
    class_data = df_raw[df_raw['class_name'] == class_name]
    axes[0, 0].hist(class_data['text_length'], alpha=0.7, label=class_name, bins=50)
axes[0, 0].set_title('Text Length Distribution (Characters)')
axes[0, 0].set_xlabel('Characters')
axes[0, 0].set_ylabel('Frequency')
axes[0, 0].legend()

# Word count distribution
for class_name in df_raw['class_name'].unique():
    class_data = df_raw[df_raw['class_name'] == class_name]
    axes[0, 1].hist(class_data['word_count'], alpha=0.7, label=class_name, bins=50)
axes[0, 1].set_title('Word Count Distribution')
axes[0, 1].set_xlabel('Words')
axes[0, 1].set_ylabel('Frequency')
axes[0, 1].legend()

# Box plots for comparison
sns.boxplot(data=df_raw, x='class_name', y='text_length', ax=axes[1, 0])
axes[1, 0].set_title('Text Length by Class (Box Plot)')
axes[1, 0].tick_params(axis='x', rotation=45)

sns.boxplot(data=df_raw, x='class_name', y='word_count', ax=axes[1, 1])
axes[1, 1].set_title('Word Count by Class (Box Plot)')
axes[1, 1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../reports/figures/text_length_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 4. Sample Texts by Class

In [ ]:
# Show sample texts for each class
print("Sample Texts by Class:")
print("=" * 60)

for class_name in ['Depression', 'Non-Depression']:
    print(f"\n{class_name.upper()} EXAMPLES:")
    print("-" * 40)
    
    class_data = df_raw[df_raw['class_name'] == class_name]
    samples = class_data.sample(n=5, random_state=42)
    
    for i, (_, row) in enumerate(samples.iterrows(), 1):
        text = row['text']
        # Truncate very long texts
        if len(text) > 150:
            text = text[:150] + "..."
        print(f"{i}. {text}")
    
    print(f"\nClass statistics:")
    print(f"  Total samples: {len(class_data):,}")
    print(f"  Avg text length: {class_data['text_length'].mean():.1f} characters")
    print(f"  Avg word count: {class_data['word_count'].mean():.1f} words")

## 5. Word Frequency Analysis

In [ ]:
# Word frequency analysis
def get_top_words(texts, n=20, min_length=2):
    """Get top N words from texts."""
    # Combine all texts
    all_text = ' '.join(texts).lower()
    
    # Extract words (remove punctuation)
    words = re.findall(r'\b[a-z]+\b', all_text)
    
    # Filter by length and common stop words
    stop_words = {'the', 'and', 'or', 'but', 'in', 'on', 'at', 'to', 'for', 'of', 'with', 'by',
                  'is', 'are', 'was', 'were', 'be', 'been', 'being', 'have', 'has', 'had',
                  'do', 'does', 'did', 'will', 'would', 'should', 'could', 'can', 'may',
                  'a', 'an', 'as', 'if', 'it', 'its', 'this', 'that', 'these', 'those'}
    
    filtered_words = [word for word in words if len(word) >= min_length and word not in stop_words]
    
    return Counter(filtered_words).most_common(n)

# Get top words for each class
depression_texts = df_raw[df_raw['class_name'] == 'Depression']['text'].tolist()
non_depression_texts = df_raw[df_raw['class_name'] == 'Non-Depression']['text'].tolist()

top_depression_words = get_top_words(depression_texts, n=20)
top_non_depression_words = get_top_words(non_depression_texts, n=20)

print("Top 20 Words by Class:")
print("\nDEPRESSION CLASS:")
for i, (word, count) in enumerate(top_depression_words, 1):
    print(f"{i:2d}. {word:15s} ({count:,} occurrences)")

print("\nNON-DEPRESSION CLASS:")
for i, (word, count) in enumerate(top_non_depression_words, 1):
    print(f"{i:2d}. {word:15s} ({count:,} occurrences)")

# Visualize top words
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(16, 8))

# Depression words
words_dep = [item[0] for item in top_depression_words[:15]]
counts_dep = [item[1] for item in top_depression_words[:15]]
ax1.barh(words_dep[::-1], counts_dep[::-1], color='#ff7f7f')
ax1.set_title('Top 15 Words - Depression Class')
ax1.set_xlabel('Frequency')

# Non-depression words
words_non_dep = [item[0] for item in top_non_depression_words[:15]]
counts_non_dep = [item[1] for item in top_non_depression_words[:15]]
ax2.barh(words_non_dep[::-1], counts_non_dep[::-1], color='#7f7fff')
ax2.set_title('Top 15 Words - Non-Depression Class')
ax2.set_xlabel('Frequency')

plt.tight_layout()
plt.savefig('../reports/figures/top_words_by_class.png', dpi=300, bbox_inches='tight')
plt.show()

## 6. Hashtag and Mention Analysis

In [ ]:
# Extract hashtags and mentions
def extract_hashtags(text):
    return re.findall(r'#\w+', text.lower())

def extract_mentions(text):
    return re.findall(r'@\w+', text.lower())

# Extract for each class
df_raw['hashtags'] = df_raw['text'].apply(extract_hashtags)
df_raw['mentions'] = df_raw['text'].apply(extract_mentions)
df_raw['hashtag_count'] = df_raw['hashtags'].str.len()
df_raw['mention_count'] = df_raw['mentions'].str.len()

# Statistics by class
social_stats = df_raw.groupby('class_name')[['hashtag_count', 'mention_count']].agg(['mean', 'std', 'sum'])
print("Hashtag and Mention Statistics by Class:")
print(social_stats.round(2))

# Top hashtags by class
print("\nTop Hashtags by Class:")
for class_name in ['Depression', 'Non-Depression']:
    class_data = df_raw[df_raw['class_name'] == class_name]
    all_hashtags = [tag for tags in class_data['hashtags'] for tag in tags]
    top_hashtags = Counter(all_hashtags).most_common(10)
    
    print(f"\n{class_name.upper()}:")
    for i, (hashtag, count) in enumerate(top_hashtags, 1):
        print(f"  {i:2d}. {hashtag:20s} ({count:,} times)")

# Visualize hashtag usage
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

# Hashtag count distribution
sns.boxplot(data=df_raw, x='class_name', y='hashtag_count', ax=axes[0])
axes[0].set_title('Hashtag Count by Class')
axes[0].tick_params(axis='x', rotation=45)

# Mention count distribution
sns.boxplot(data=df_raw, x='class_name', y='mention_count', ax=axes[1])
axes[1].set_title('Mention Count by Class')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../reports/figures/social_features_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 7. Sentiment and Feature Analysis

In [ ]:
# Extract additional features using our FeatureExtractor
feature_extractor = FeatureExtractor()

print("Extracting additional features...")
feature_data = []

# Sample a subset for performance (extract features for 1000 samples per class)
sample_df = df_raw.groupby('class_name').apply(lambda x: x.sample(n=min(1000, len(x)), random_state=42)).reset_index(drop=True)

for _, row in sample_df.iterrows():
    features = feature_extractor.extract_all_features(row['text'])
    features['class_name'] = row['class_name']
    features['label'] = row['label']
    feature_data.append(features)

feature_df = pd.DataFrame(feature_data)

print(f"Extracted features for {len(feature_df)} samples")
print("\nFeature columns:", feature_df.columns.tolist())

# Feature statistics by class
feature_cols = ['char_count', 'word_count', 'sentence_count', 'avg_word_length', 
               'exclamation_count', 'question_count', 'uppercase_ratio',
               'sentiment_compound', 'sentiment_positive', 'sentiment_negative']

print("\nFeature Statistics by Class:")
feature_stats = feature_df.groupby('class_name')[feature_cols].mean()
print(feature_stats.round(3))

In [ ]:
# Visualize key features
fig, axes = plt.subplots(2, 3, figsize=(18, 12))
axes = axes.ravel()

# Features to plot
plot_features = [
    ('sentiment_compound', 'Sentiment (Compound Score)'),
    ('sentiment_negative', 'Negative Sentiment'),
    ('sentiment_positive', 'Positive Sentiment'),
    ('exclamation_count', 'Exclamation Marks'),
    ('question_count', 'Question Marks'),
    ('uppercase_ratio', 'Uppercase Ratio')
]

for i, (feature, title) in enumerate(plot_features):
    if feature in feature_df.columns:
        sns.boxplot(data=feature_df, x='class_name', y=feature, ax=axes[i])
        axes[i].set_title(title)
        axes[i].tick_params(axis='x', rotation=45)
    else:
        axes[i].text(0.5, 0.5, f'{feature}\nnot available', 
                    ha='center', va='center', transform=axes[i].transAxes)
        axes[i].set_title(f'{title} (N/A)')

plt.tight_layout()
plt.savefig('../reports/figures/feature_analysis.png', dpi=300, bbox_inches='tight')
plt.show()

## 8. Text Complexity and Readability

In [ ]:
# Analyze text complexity
def flesch_reading_ease(text):
    """Simple approximation of Flesch Reading Ease score."""
    sentences = len(re.findall(r'[.!?]+', text))
    words = len(text.split())
    syllables = sum([len(re.findall(r'[aeiouAEIOU]', word)) for word in text.split()])
    
    if sentences == 0 or words == 0:
        return 0
    
    asl = words / sentences  # Average sentence length
    asw = syllables / words if words > 0 else 0  # Average syllables per word
    
    fre = 206.835 - (1.015 * asl) - (84.6 * asw)
    return max(0, min(100, fre))

# Calculate readability for sample
sample_df['readability'] = sample_df['text'].apply(flesch_reading_ease)
sample_df['avg_word_length'] = sample_df['text'].apply(lambda x: np.mean([len(word) for word in x.split()]) if x.split() else 0)

# Compare complexity between classes
complexity_stats = sample_df.groupby('class_name')[['readability', 'avg_word_length']].describe()
print("Text Complexity by Class:")
print(complexity_stats.round(2))

# Visualize complexity
fig, axes = plt.subplots(1, 2, figsize=(14, 6))

sns.boxplot(data=sample_df, x='class_name', y='readability', ax=axes[0])
axes[0].set_title('Text Readability Score by Class')
axes[0].set_ylabel('Flesch Reading Ease Score')
axes[0].tick_params(axis='x', rotation=45)

sns.boxplot(data=sample_df, x='class_name', y='avg_word_length', ax=axes[1])
axes[1].set_title('Average Word Length by Class')
axes[1].set_ylabel('Characters per Word')
axes[1].tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.savefig('../reports/figures/text_complexity.png', dpi=300, bbox_inches='tight')
plt.show()

## 9. TF-IDF and Dimensionality Reduction Visualization

In [ ]:
# TF-IDF and t-SNE visualization
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

print("Creating TF-IDF vectors...")

# Use a smaller sample for t-SNE (computationally expensive)
viz_sample = df_raw.groupby('class_name').apply(lambda x: x.sample(n=min(500, len(x)), random_state=42)).reset_index(drop=True)

# Create TF-IDF vectors
tfidf = TfidfVectorizer(
    max_features=1000,
    ngram_range=(1, 2),
    stop_words='english',
    min_df=2,
    max_df=0.95
)

tfidf_matrix = tfidf.fit_transform(viz_sample['text'])
print(f"TF-IDF matrix shape: {tfidf_matrix.shape}")

# Get feature names
feature_names = tfidf.get_feature_names_out()
print(f"Example features: {feature_names[:10]}")

In [ ]:
# Dimensionality reduction with PCA first (faster)
print("Applying PCA for dimensionality reduction...")
pca = PCA(n_components=50, random_state=42)
tfidf_pca = pca.fit_transform(tfidf_matrix.toarray())

print(f"Explained variance ratio (first 10 components): {pca.explained_variance_ratio_[:10].round(3)}")
print(f"Total explained variance (50 components): {pca.explained_variance_ratio_.sum():.3f}")

# Apply t-SNE
print("Applying t-SNE...")
tsne = TSNE(n_components=2, random_state=42, perplexity=30, n_iter=1000)
tfidf_tsne = tsne.fit_transform(tfidf_pca)

# Create visualization DataFrame
viz_df = pd.DataFrame({
    'x': tfidf_tsne[:, 0],
    'y': tfidf_tsne[:, 1],
    'class_name': viz_sample['class_name'].values,
    'label': viz_sample['label'].values
})

print("t-SNE visualization complete!")

In [ ]:
# Visualize t-SNE results
plt.figure(figsize=(12, 8))

# Create scatter plot
colors = ['#ff7f7f', '#7f7fff']
for i, class_name in enumerate(['Depression', 'Non-Depression']):
    class_data = viz_df[viz_df['class_name'] == class_name]
    plt.scatter(class_data['x'], class_data['y'], 
               c=colors[i], label=class_name, alpha=0.7, s=30)

plt.title('t-SNE Visualization of Tweet TF-IDF Vectors\n(Depression vs Non-Depression)', fontsize=14)
plt.xlabel('t-SNE Dimension 1')
plt.ylabel('t-SNE Dimension 2')
plt.legend()
plt.grid(True, alpha=0.3)

# Add sample size info
plt.figtext(0.02, 0.02, f'Sample size: {len(viz_df)} tweets ({len(viz_df[viz_df.label==1])} Depression, {len(viz_df[viz_df.label==0])} Non-Depression)',
           fontsize=10, style='italic')

plt.tight_layout()
plt.savefig('../reports/figures/tsne_visualization.png', dpi=300, bbox_inches='tight')
plt.show()

print("\nt-SNE Interpretation:")
print("- Points close together have similar TF-IDF patterns")
print("- Clear separation suggests classes use different vocabulary")
print("- Overlap areas might be challenging for classification")

## 10. Most Discriminative Features

In [ ]:
# Find most discriminative TF-IDF features
from sklearn.feature_selection import chi2

# Calculate chi-squared scores for feature selection
chi2_scores, p_values = chi2(tfidf_matrix, viz_sample['label'])

# Create feature importance dataframe
feature_scores = pd.DataFrame({
    'feature': feature_names,
    'chi2_score': chi2_scores,
    'p_value': p_values
}).sort_values('chi2_score', ascending=False)

# Top discriminative features
print("Top 20 Most Discriminative Features (Chi-squared test):")
print("=" * 55)
for i, (_, row) in enumerate(feature_scores.head(20).iterrows(), 1):
    print(f"{i:2d}. {row['feature']:25s} (χ² = {row['chi2_score']:8.2f}, p = {row['p_value']:.2e})")

# Visualize top features
top_features = feature_scores.head(15)

plt.figure(figsize=(12, 8))
plt.barh(range(len(top_features)), top_features['chi2_score'], color='skyblue')
plt.yticks(range(len(top_features)), top_features['feature'])
plt.xlabel('Chi-squared Score')
plt.title('Top 15 Most Discriminative TF-IDF Features')
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)

plt.tight_layout()
plt.savefig('../reports/figures/discriminative_features.png', dpi=300, bbox_inches='tight')
plt.show()

## 11. Generate EDA Report Summary

In [ ]:
# Create comprehensive EDA report
eda_report = {
    'analysis_date': '2025-09-30',
    'dataset_overview': {
        'total_samples': len(df_raw),
        'classes': df_raw['class_name'].value_counts().to_dict(),
        'class_balance_ratio': df_raw['class_name'].value_counts().max() / df_raw['class_name'].value_counts().min(),
        'imbalanced': (df_raw['class_name'].value_counts().max() / df_raw['class_name'].value_counts().min()) > 2
    },
    'text_statistics': {
        'avg_text_length': df_raw['text_length'].mean(),
        'avg_word_count': df_raw['word_count'].mean(),
        'text_length_by_class': df_raw.groupby('class_name')['text_length'].mean().to_dict(),
        'word_count_by_class': df_raw.groupby('class_name')['word_count'].mean().to_dict()
    },
    'vocabulary_analysis': {
        'top_depression_words': dict(top_depression_words[:10]),
        'top_non_depression_words': dict(top_non_depression_words[:10]),
        'most_discriminative_features': feature_scores.head(10)[['feature', 'chi2_score']].to_dict('records')
    },
    'social_features': {
        'avg_hashtags_by_class': df_raw.groupby('class_name')['hashtag_count'].mean().to_dict(),
        'avg_mentions_by_class': df_raw.groupby('class_name')['mention_count'].mean().to_dict()
    },
    'complexity_analysis': {
        'readability_by_class': sample_df.groupby('class_name')['readability'].mean().to_dict(),
        'avg_word_length_by_class': sample_df.groupby('class_name')['avg_word_length'].mean().to_dict()
    },
    'key_findings': [
        f"Dataset contains {len(df_raw):,} tweets with {'im' if (df_raw['class_name'].value_counts().max() / df_raw['class_name'].value_counts().min()) > 2 else ''}balanced classes",
        f"Depression tweets average {df_raw[df_raw.class_name=='Depression']['text_length'].mean():.1f} characters vs {df_raw[df_raw.class_name=='Non-Depression']['text_length'].mean():.1f} for non-depression",
        f"Most discriminative feature: '{feature_scores.iloc[0]['feature']}' (χ² = {feature_scores.iloc[0]['chi2_score']:.1f})",
        "t-SNE visualization shows distinguishable clustering patterns between classes"
    ],
    'recommendations': [
        "Use TF-IDF features as they show clear discriminative patterns",
        "Consider class weighting due to imbalance" if (df_raw['class_name'].value_counts().max() / df_raw['class_name'].value_counts().min()) > 2 else "Classes are reasonably balanced",
        "Include sentiment and social features (hashtags, mentions) in model",
        "Text preprocessing should preserve emotional indicators while removing noise"
    ]
}

# Save report as JSON
with open('../reports/eda_report.json', 'w') as f:
    json.dump(eda_report, f, indent=2, default=str)

print("EDA Report Summary:")
print("=" * 50)
for finding in eda_report['key_findings']:
    print(f"• {finding}")

print("\nRecommendations:")
print("-" * 30)
for rec in eda_report['recommendations']:
    print(f"• {rec}")

print(f"\n✅ Complete EDA report saved to: reports/eda_report.json")
print(f"✅ All visualizations saved to: reports/figures/")

In [ ]:
# Generate HTML report
html_content = f"""
<!DOCTYPE html>
<html>
<head>
    <title>Mental Health Tweet Classification - EDA Report</title>
    <style>
        body {{ font-family: Arial, sans-serif; margin: 40px; line-height: 1.6; }}
        .header {{ background-color: #f4f4f4; padding: 20px; border-radius: 10px; }}
        .section {{ margin: 30px 0; }}
        .metric {{ background-color: #e8f4f8; padding: 15px; margin: 10px 0; border-radius: 5px; }}
        .finding {{ background-color: #f0f8e8; padding: 10px; margin: 5px 0; border-radius: 3px; }}
        .recommendation {{ background-color: #fff8e0; padding: 10px; margin: 5px 0; border-radius: 3px; }}
        table {{ border-collapse: collapse; width: 100%; margin: 10px 0; }}
        th, td {{ border: 1px solid #ddd; padding: 8px; text-align: left; }}
        th {{ background-color: #f2f2f2; }}
    </style>
</head>
<body>
    <div class="header">
        <h1>Mental Health Tweet Classification</h1>
        <h2>Exploratory Data Analysis Report</h2>
        <p><strong>Date:</strong> {eda_report['analysis_date']}</p>
        <p><strong>Dataset:</strong> Depression vs Non-Depression Tweet Classification</p>
    </div>

    <div class="section">
        <h3>Dataset Overview</h3>
        <div class="metric">
            <strong>Total Samples:</strong> {eda_report['dataset_overview']['total_samples']:,}<br>
            <strong>Classes:</strong> {eda_report['dataset_overview']['classes']}<br>
            <strong>Class Balance Ratio:</strong> {eda_report['dataset_overview']['class_balance_ratio']:.2f}<br>
            <strong>Imbalanced:</strong> {eda_report['dataset_overview']['imbalanced']}
        </div>
    </div>

    <div class="section">
        <h3>Key Findings</h3>
        {''.join([f'<div class="finding">• {finding}</div>' for finding in eda_report['key_findings']])}
    </div>

    <div class="section">
        <h3>Recommendations</h3>
        {''.join([f'<div class="recommendation">• {rec}</div>' for rec in eda_report['recommendations']])}
    </div>

    <div class="section">
        <h3>Text Statistics</h3>
        <table>
            <tr><th>Metric</th><th>Depression</th><th>Non-Depression</th></tr>
            <tr><td>Avg Text Length</td><td>{eda_report['text_statistics']['text_length_by_class']['Depression']:.1f}</td><td>{eda_report['text_statistics']['text_length_by_class']['Non-Depression']:.1f}</td></tr>
            <tr><td>Avg Word Count</td><td>{eda_report['text_statistics']['word_count_by_class']['Depression']:.1f}</td><td>{eda_report['text_statistics']['word_count_by_class']['Non-Depression']:.1f}</td></tr>
        </table>
    </div>

    <div class="section">
        <h3>Generated Visualizations</h3>
        <ul>
            <li>Class Distribution (figures/class_distribution.png)</li>
            <li>Text Length Analysis (figures/text_length_analysis.png)</li>
            <li>Top Words by Class (figures/top_words_by_class.png)</li>
            <li>Social Features Analysis (figures/social_features_analysis.png)</li>
            <li>Feature Analysis (figures/feature_analysis.png)</li>
            <li>Text Complexity (figures/text_complexity.png)</li>
            <li>t-SNE Visualization (figures/tsne_visualization.png)</li>
            <li>Discriminative Features (figures/discriminative_features.png)</li>
        </ul>
    </div>

</body>
</html>
"""

# Save HTML report
with open('../reports/eda_report.html', 'w') as f:
    f.write(html_content)

print("✅ HTML report generated: reports/eda_report.html")
print("\n🎉 Exploratory Data Analysis Complete!")
print("\nNext steps:")
print("1. Review the generated visualizations and reports")
print("2. Proceed with feature engineering and model building")
print("3. Use insights from EDA to guide preprocessing and model selection")